### V2 Recommendations

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
from calitp_data_analysis.sql import to_snakecase

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
import _threshold_utils as threshold_utils

In [3]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [4]:
threshold_utils.create_operator_visuals(500,400, threshold_utils.TIME_CUTOFFS, threshold_utils.SEGMENT_CUTOFFS)

alt.VConcatChart(...)

In [5]:
route_length, time_segments, valid_stats = threshold_utils.load_dataframes(threshold_utils.TIME_CUTOFFS, threshold_utils.SEGMENT_CUTOFFS)

In [6]:
threshold_utils.operator_brush(route_length)

alt.VConcatChart(...)

### Statewide
#### Statewide Routes by Cutoff

In [7]:
threshold_utils.create_statewide_visuals(500,400)

alt.VConcatChart(...)

### Recs - 15 minutes, 70% of segments

In [8]:
time_segments.trip_time.median()

59.0

In [9]:
leniency_strigency = threshold_utils.valid_stats_leniency(valid_stats)

In [10]:
f"Median % of trips kept if the most lenient threshold is applied {leniency_strigency.Max.median()}%"

'Median % of trips kept if the most lenient threshold is applied 98.7%'

In [11]:
f"Median % of trips kept if the most stringent threshold is applied {leniency_strigency.Min.median()}%"

'Median % of trips kept if the most stringent threshold is applied 83.7%'

#### Find specific missing routes after applying thresholds

In [12]:
threshold_utils.find_cut_routes(15, 0.7)

Empty DataFrame
Columns: []
Index: [(Bay Area 511 Santa Clara Transit Schedule, 03cadbffef6f3cd4ce839b218bc65e82, ACE Gray), (Bay Area 511 Santa Clara Transit Schedule, 03cadbffef6f3cd4ce839b218bc65e82, ACE Green), (Bay Area 511 Santa Clara Transit Schedule, 03cadbffef6f3cd4ce839b218bc65e82, ACE Orange), (Bay Area 511 Santa Clara Transit Schedule, 03cadbffef6f3cd4ce839b218bc65e82, ACE Purple), (Bay Area 511 Santa Clara Transit Schedule, 03cadbffef6f3cd4ce839b218bc65e82, ACE Red), (Bay Area 511 Santa Clara Transit Schedule, 03cadbffef6f3cd4ce839b218bc65e82, ACE Violet), (Bay Area 511 Vine Transit Schedule, 82d94402b3325d1e93e2a1f5b80a300d, S), (Eastern Sierra Schedule, ae93720f3c32c12f9ed912e465066bfc, 766), (Humboldt Schedule, ee4a7698aa24503c4058d10c238e6f8a, 18689), (Lake Schedule, 087e5ecdcf65499829e681a60e606637, 2033), (SBMTD Schedule, c5b2bf3815a12d5f7242923ed399d1ea, 2520), (SBMTD Schedule, c5b2bf3815a12d5f7242923ed399d1ea, 2610), (VCTC GMV Schedule, 910fbc31da89056bdee9ef48b235ef44, 3639)]